# Задача
- На основе тренировочного множества (classification_train.csv) постройте модель, которая сможет определять по входных данных (все столбцы, кроме target),  произойдет событие (target = 1) или нет (target = 0). 
- На основе разработанной модели составьте прогноз для тестового множества (classification_test.csv).  
- Приложите описание решения (код или какие-то другие материалы, созданные в ходе решения приветствуются). 
- Приложите прогноз в виде, представленном в sample submission.csv

In [1]:
! ls -l ./data

total 17920
-rw-rw-r--@ 1 Evgeny  staff  1824443 Aug 22  2016 classification_test.csv
-rw-rw-r--@ 1 Evgeny  staff  7343138 Aug 22  2016 classification_train.csv
-rw-rw-r--@ 1 Evgeny  staff       38 Aug 22  2016 sample submission.csv


# Looking at data and preparing

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [3]:
df = pd.read_csv('./data/classification_train.csv')
df_test = pd.read_csv('./data/classification_test.csv')

In [4]:
df.shape[0]
df[:5]

18420

,row_number,day_of_week,time_interval,numerical_feature1,binary_feature1,numerical_feature2,numerical_feature3,numerical_feature4,numerical_feature5,numerical_feature6,numerical_feature7,numerical_feature8,numerical_feature9,numerical_feature10,numerical_feature11,numerical_feature12,numerical_feature13,numerical_feature14,numerical_feature15,numerical_feature16,numerical_feature17,numerical_feature18,numerical_feature19,numerical_feature20,numerical_feature21,target
0,1,6,28,-1.293887,0,-0.202881,-0.148187,-0.915432,0.123721,0.067026,-0.174143,-0.679518,-0.419538,-0.085589,-0.907569,0.158600,0.080291,-0.197479,-0.756770,-0.410152,-1.145737,-1.135511,-0.368707,-0.692058,-0.176466,1
1,2,5,9,2.046774,0,-0.202881,-0.429073,-1.381668,-0.153588,0.105036,0.340381,-0.358484,-1.187368,-0.225571,-1.322025,-0.036902,0.285351,0.574601,-0.367721,-1.378392,-0.797372,-0.857479,-0.315709,0.240003,-0.176466,0
2,3,6,13,1.789800,0,-0.202881,-0.268481,0.867711,-0.497171,-1.075316,-1.518250,0.797099,0.959080,-0.951920,0.394999,-1.006386,-1.337036,-0.733739,0.712635,0.309325,0.910619,0.606608,-0.594949,-0.651534,1.855187,0
3,4,7,19,0.504930,0,-0.202881,-0.619177,1.318468,-2.109093,-1.114104,0.327221,1.722900,0.683492,-1.416160,0.122735,-1.148890,-1.040386,-0.275062,0.653164,-0.116479,1.716941,1.326551,0.015649,0.158954,3.886840,0
4,5,3,9,-0.522965,0,-0.202881,-1.158597,-0.353492,-0.797533,-0.786577,-0.456209,0.208238,-0.321273,-1.166933,-0.011053,-0.923628,-0.995696,-0.493787,0.445493,-0.090425,-0.569411,-0.705167,-0.485654,-0.124716,-0.176466,1


In [5]:
df.corr()['target'].abs().sort_values(ascending=False)[:5]

target                 1.000000
numerical_feature1     0.350570
numerical_feature17    0.261505
numerical_feature18    0.260588
numerical_feature21    0.162189
Name: target, dtype: float64

In [6]:
df.describe()
df_test.describe()

/Users/Evgeny/.virtualenvs/datascience/lib/python3.5/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,row_number,day_of_week,time_interval,numerical_feature1,binary_feature1,numerical_feature2,numerical_feature3,numerical_feature4,numerical_feature5,numerical_feature6,numerical_feature7,numerical_feature8,numerical_feature9,numerical_feature10,numerical_feature11,numerical_feature12,numerical_feature13,numerical_feature14,numerical_feature15,numerical_feature16,numerical_feature17,numerical_feature18,numerical_feature19,numerical_feature20,numerical_feature21,target
count,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18420.000000,18419.000000,18419.000000,18419.000000,18419.000000,18419.000000,18419.000000,18419.000000,18419.000000,18419.000000,18419.000000,18420.000000,18420.000000,18420.000000
mean,9210.500000,3.964495,15.626059,-0.004637,0.040011,-0.000295,0.003799,0.000736,0.005228,0.004205,0.002685,-0.002729,0.001871,0.006395,-0.000676,0.008141,0.006883,0.003906,-0.004815,0.000463,-0.005381,-0.005341,-0.002344,0.000445,-0.006500,0.499077
std,5317.540315,1.872689,10.576227,1.000443,0.195990,0.999527,1.008043,1.003409,1.005749,1.005374,1.001369,1.000152,1.004276,1.007227,1.004028,1.005808,1.006305,1.001639,0.998324,1.007471,0.997451,0.997268,1.007052,1.000764,0.987563,0.500013
min,1.000000,1.000000,0.000000,-1.807835,0.000000,-0.202881,-1.789510,-1.839408,-2.283283,-1.980911,-1.870994,-2.092420,-2.572195,-1.560905,-1.856822,-2.111801,-1.935753,-1.936510,-2.413073,-2.551180,-1.398410,-1.394246,-0.677634,-1.543071,-0.176466,0.000000
25%,4605.750000,2.000000,6.000000,-0.779939,0.000000,-0.202881,-0.690041,-0.741257,-0.707542,-0.770828,-0.694720,-0.643349,-0.492581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.651534,-0.176466,0.000000
50%,9210.500000,4.000000,14.000000,-0.265991,0.000000,-0.202881,-0.094823,-0.162567,-0.036937,-0.043635,-0.114775,0.029001,-0.083286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.286814,-0.176466,0.000000
75%,13815.250000,5.000000,25.000000,0.504930,0.000000,-0.202881,0.597771,0.665734,0.668696,0.719044,0.634048,0.680481,0.575705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.402101,-0.176466,1.000000
max,18420.000000,7.000000,35.000000,5.901382,1.000000,9.879596,17.882610,18.408528,15.934948,15.604594,15.208451,13.258007,17.480824,17.169138,18.733943,15.795156,16.117432,16.000404,14.560901,17.859575,4.846631,4.758302,19.072898,6.197090,20.140064,1.000000


,row_number,day_of_week,time_interval,numerical_feature1,binary_feature1,numerical_feature2,numerical_feature3,numerical_feature4,numerical_feature5,numerical_feature6,numerical_feature7,numerical_feature8,numerical_feature9,numerical_feature10,numerical_feature11,numerical_feature12,numerical_feature13,numerical_feature14,numerical_feature15,numerical_feature16,numerical_feature17,numerical_feature18,numerical_feature19,numerical_feature20,numerical_feature21
count,4606.000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4606.000000,4606.000000
mean,2303.500,3.985888,15.597482,0.018543,0.040599,0.001181,-0.015193,-0.002943,-0.020909,-0.016817,-0.010739,0.010915,-0.007482,-0.025579,0.002702,-0.032563,-0.027532,-0.015625,0.019260,-0.001853,0.021521,0.021362,0.009375,-0.001780,0.025994
std,1329.782,1.895883,10.669955,0.998119,0.197381,1.001998,0.967124,0.986352,0.976500,0.978139,0.994544,0.999424,0.982788,0.970242,0.983827,0.975852,0.973995,0.993374,1.006554,0.969646,1.009953,1.010681,0.971333,0.997044,1.047970
min,1.000,1.000000,0.000000,-1.807835,0.000000,-0.202881,-1.788199,-1.828044,-2.283283,-1.978166,-1.854289,-2.053969,-2.499387,-1.550014,-1.834461,-2.119922,-1.912597,-1.948652,-2.385950,-2.540920,-1.397812,-1.393617,-0.635016,-1.543071,-0.176466
25%,1152.250,2.000000,6.000000,-0.779939,0.000000,-0.202881,-0.691775,-0.749689,-0.715963,-0.766546,-0.715866,-0.638647,-0.514883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.651534,-0.176466
50%,2303.500,4.000000,14.000000,-0.009017,0.000000,-0.202881,-0.103263,-0.170131,-0.048276,-0.081485,-0.115258,0.054411,-0.094082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.286814,-0.176466
75%,3454.750,5.000000,25.000000,0.504930,0.000000,-0.202881,0.588268,0.654850,0.639534,0.693918,0.607256,0.680481,0.587147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.402101,-0.176466
max,4606.000,7.000000,35.000000,5.387434,1.000000,10.626446,5.201711,4.928027,4.717585,5.279611,5.570260,4.704657,4.775193,5.293866,5.481027,4.966087,5.688757,5.998763,5.160179,5.547135,4.708229,4.528985,10.021254,6.197090,20.140064


we see that for some of features there NA in one line. As it's relatively rare case let's jst fill na with mean value

In [7]:
df = df.fillna(df.mean())
df_test = df_test.fillna(df_test.mean())

looks like we have 2 outliers in train set with index 4296 and 39. Let's remove them

In [8]:
#
for i in range(1,22):    
    df['numerical_feature{}'.format(i)].sort_values(ascending=0)[:3]

4885    5.901382
5867    5.901382
9626    5.644408
Name: numerical_feature1, dtype: float64

8370    9.879596
5048    8.759321
23      8.012470
Name: numerical_feature2, dtype: float64

4296    17.882610
39      10.800253
8746     6.840987
Name: numerical_feature3, dtype: float64

4296     18.408528
39       11.323055
17275     6.393609
Name: numerical_feature4, dtype: float64

4296    15.934948
39       9.462586
8746     6.369004
Name: numerical_feature5, dtype: float64

...

12926    4.846631
287      4.642878
18238    4.610173
Name: numerical_feature17, dtype: float64

13032    4.758302
13770    4.582166
8521     4.504659
Name: numerical_feature18, dtype: float64

2739    19.072898
9677    17.807809
8325    14.807909
Name: numerical_feature19, dtype: float64

10154    6.19709
3377     6.19709
13360    6.19709
Name: numerical_feature20, dtype: float64

4598     20.140064
18012    16.076758
8710     16.076758
Name: numerical_feature21, dtype: float64

In [9]:
df = df.sort_values(by='numerical_feature3', ascending=False)[2:]

According to names and distribution **row_number** and **day_of_week** are candidates for one hot encoding

In [10]:
# encode train set
df = pd.get_dummies(df, prefix='wd_', columns=['day_of_week'])
df = pd.get_dummies(df, prefix='ti_', columns=['time_interval'])
# encode test set
df_test = pd.get_dummies(df_test, prefix='wd_', columns=['day_of_week'])
df_test = pd.get_dummies(df_test, prefix='ti_', columns=['time_interval'])

In [25]:
# answers are distributed almost equally
df.target.sum()/df.shape[0]

0.4990769899011836

#  Checking models

In [87]:
from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from scipy.stats import randint as sp_randint

import scipy
from sklearn.ensemble import RandomForestClassifier

# get some dataimport numpy as np

In [39]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [72]:
def save_result(fname, arr):
    with open('data/'+fname, 'w') as f:
        f.write('row_number,prediction\n')
        c=1
        for val in arr:
            f.write('{},{}\n'.format(c, val))
            c += 1

In [33]:
X = df.drop(labels=['target'], axis=1)
Y = df.target

X_test = df_test

In [34]:
# scaling
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_test = scaler.transform(df_test)

### SVM

In [66]:
X_train, X_dev, Y_train, Y_dev = train_test_split(
    X, Y, test_size=0.2, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


# specify parameters and distributions to sample from
param_dist = {"kernel": ['linear', 'rbf'],
              'C': scipy.stats.expon(scale=100), 
              'gamma': scipy.stats.expon(scale=.1)
              }

clf = RandomizedSearchCV(svm.SVC(), param_distributions=param_dist, n_iter=10,verbose=2, n_jobs=-1)
# clf = GridSearchCV(svm.SVC(C=1), tuned_parameters, cv=5,verbose=2, n_jobs=-1)
clf.fit(X_train, Y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] gamma=0.464828139913, C=136.358217257, kernel=linear ............
[CV] gamma=0.464828139913, C=136.358217257, kernel=linear ............
[CV] gamma=0.464828139913, C=136.358217257, kernel=linear ............
[CV] gamma=0.002108355201, C=88.8959157483, kernel=rbf ...............
[CV] ...... gamma=0.002108355201, C=88.8959157483, kernel=rbf -   4.2s
[CV] gamma=0.002108355201, C=88.8959157483, kernel=rbf ...............
[CV] ...... gamma=0.002108355201, C=88.8959157483, kernel=rbf -   4.1s
[CV] gamma=0.002108355201, C=88.8959157483, kernel=rbf ...............
[CV] ...... gamma=0.002108355201, C=88.8959157483, kernel=rbf -   4.4s
[CV] gamma=0.00503160069854, C=65.0743476334, kernel=rbf .............
[CV] .... gamma=0.00503160069854, C=65.0743476334, kernel=rbf -   4.2s
[CV] gamma=0.00503160069854, C=65.0743476334, kernel=rbf .............
[CV] .... gamma=0.00503160069854, C=65.0743476334, kernel=rbf -   4.1s
[CV] gamma=0.005

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 51.5min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10827d438>, 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x114e292e8>, 'kernel': ['linear', 'rbf']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [68]:
Y_true, Y_pred = Y_dev, clf.predict(X_dev)

print(classification_report(Y_true, Y_pred))

print(metrics.accuracy_score(Y_true, Y_pred) )

             precision    recall  f1-score   support

          0       0.70      0.61      0.65      1806
          1       0.67      0.75      0.71      1878

avg / total       0.68      0.68      0.68      3684

0.681867535288


In [70]:
pred = clf.predict(X_test)
pred

array([0, 0, 1, ..., 1, 0, 1])

In [73]:
save_result('SVM_RS.csv', pred)

### RandomForest

In [80]:
X_train, X_dev, Y_train, Y_dev = train_test_split(
    X, Y, test_size=0.2, random_state=31)
clf_rf = RandomForestClassifier(n_estimators=20)

In [81]:
clf_rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [82]:
Y_true, Y_pred = Y_dev, clf_rf.predict(X_dev)
metrics.accuracy_score(Y_true, Y_pred)

0.66775244299674263

In [83]:
pred = clf_rf.predict(X_test)
pred

array([1, 0, 0, ..., 1, 0, 1])

In [84]:
save_result('RF.csv', pred)

### Random Forest random search

In [85]:
X_train, X_dev, Y_train, Y_dev = train_test_split(
    X, Y, test_size=0.2, random_state=32)

In [101]:
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False]}

# run randomized search
clf_rf_rs = RandomizedSearchCV(RandomForestClassifier(n_estimators=50), param_distributions=param_dist,
                                   n_iter=20, verbose=2)

In [102]:
clf_rf_rs.fit(X_train, Y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_depth=None, min_samples_leaf=6, bootstrap=True, min_samples_split=2, max_features=3 
[CV]  max_depth=None, min_samples_leaf=6, bootstrap=True, min_samples_split=2, max_features=3 -   0.0s
[CV] max_depth=None, min_samples_leaf=6, bootstrap=True, min_samples_split=2, max_features=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  max_depth=None, min_samples_leaf=6, bootstrap=True, min_samples_split=2, max_features=3 -   0.0s
[CV] max_depth=None, min_samples_leaf=6, bootstrap=True, min_samples_split=2, max_features=3 
[CV]  max_depth=None, min_samples_leaf=6, bootstrap=True, min_samples_split=2, max_features=3 -   0.0s
[CV] max_depth=None, min_samples_leaf=1, bootstrap=False, min_samples_split=6, max_features=9 
[CV]  max_depth=None, min_samples_leaf=1, bootstrap=False, min_samples_split=6, max_features=9 -   0.1s
[CV] max_depth=None, min_samples_leaf=1, bootstrap=False, min_samples_split=6, max_features=9 
[CV]  max_depth=None, min_samples_leaf=1, bootstrap=False, min_samples_split=6, max_features=9 -   0.1s
[CV] max_depth=None, min_samples_leaf=1, bootstrap=False, min_samples_split=6, max_features=9 
[CV]  max_depth=None, min_samples_leaf=1, bootstrap=False, min_samples_split=6, max_features=9 -   0.1s
[CV] max_depth=None, min_samples_leaf=9, bootstrap=True, min_samples_split=5, max_features=8 
[CV]  max

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.1min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'bootstrap': [True, False], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10828a080>, 'max_depth': [3, None], 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x109a88320>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x109a88400>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [103]:
Y_true, Y_pred = Y_dev, clf_rf_rs.predict(X_dev)
metrics.accuracy_score(Y_true, Y_pred)

0.68675352877307272

In [104]:
pred = clf_rf_rs.predict(X_test)
pred

array([1, 0, 0, ..., 0, 0, 1])

In [ ]:
save_result('RF_RS_e50.csv', pred)

# Keras

In [106]:
from keras.layers import Dense, Activation
from keras.models import Sequential
import keras

In [107]:
X_train, X_dev, Y_train, Y_dev = train_test_split(
    X, Y, test_size=0.2, random_state=0, random_state=33)

SyntaxError: keyword argument repeated (<ipython-input-107-2cadf7d0383b>, line 2)

In [108]:
model = Sequential()
model.add(Dense(128,input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
# model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
adam = keras.optimizers.Adam(lr=0.00001)
model.compile(optimizer=adam,
              loss='mse',
              metrics=['accuracy'])

In [114]:
model.fit(X_train, 
          Y_train, 
          nb_epoch=40, batch_size=32, validation_data=[X_dev, Y_dev])

Train on 14734 samples, validate on 3684 samples
Epoch 1/40
14734/14734 [==============================] - 2s - loss: 0.1987 - acc: 0.6926 - val_loss: 0.2126 - val_acc: 0.6667
Epoch 2/40
14734/14734 [==============================] - 2s - loss: 0.1984 - acc: 0.6929 - val_loss: 0.2125 - val_acc: 0.6656
Epoch 3/40
14734/14734 [==============================] - 2s - loss: 0.1982 - acc: 0.6941 - val_loss: 0.2124 - val_acc: 0.6669
Epoch 4/40
14734/14734 [==============================] - 2s - loss: 0.1979 - acc: 0.6948 - val_loss: 0.2124 - val_acc: 0.6664
Epoch 5/40
14734/14734 [==============================] - 2s - loss: 0.1976 - acc: 0.6947 - val_loss: 0.2123 - val_acc: 0.6659
Epoch 6/40
14734/14734 [==============================] - 2s - loss: 0.1974 - acc: 0.6946 - val_loss: 0.2123 - val_acc: 0.6650
Epoch 7/40
14734/14734 [==============================] - 2s - loss: 0.1972 - acc: 0.6955 - val_loss: 0.2122 - val_acc: 0.6661
Epoch 8/40
14734/14734 [==============================] - 2s -

In [115]:
pred = model.predict(X_test)
pred

array([[ 0.43490097],
       [ 0.15489343],
       [ 0.65762055],
       ..., 
       [ 0.583318  ],
       [ 0.00972962],
       [ 0.69474638]], dtype=float32)

In [116]:
save_result('MLP.csv', pred)